[View in Colaboratory](https://colab.research.google.com/github/tcglarry/geo/blob/master/geo_Unet_colab_correct.ipynb)

In [0]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

gpg: keybox '/tmp/tmpuq75fwmq/pubring.gpg' created
gpg: /tmp/tmpuq75fwmq/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [0]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [1]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option
aoi_aoi_test1.ipynb  TAHU PowerPoint Template.pdf
aoi_test	     TAHU PowerPoint Template - 柱狀圖 1.ods
Colab Notebooks      test for excel
PiXNet		     Untitled0.ipynb
Sample upload.txt


In [2]:

import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import subprocess
import os
import pickle
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate, Conv2DTranspose
from keras.models import Model, load_model
import pandas as pd
import sklearn 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import regularizers

Using TensorFlow backend.


In [0]:
src = 'drive/aoi_test/t_brain/geo/'

In [4]:
data= np.load(src+'train_data.npy')
label = np.load(src+'train_labels.npy')
print (np.mean(label))
label = label[:,:,:,np.newaxis]
print (data.shape)
print (label.shape)

5.66184075511905
(4200, 21, 21, 20)
(4200, 20, 20, 1)


In [0]:

data_mean = np.mean(data)
data_std = np.std(data)
label_max = np.max(label)
label_min = np.min(label)

train_data = (data - data_mean)/data_std
#label_data = (label-label_min)/(label_max - label_min)
label_data = np.log(label)

In [0]:
train_x,val_x,train_y,val_y = train_test_split(train_data,label_data,test_size=0.2, shuffle=True)

In [0]:
def conv_block(ch, activation= 'relu', padding='same',kernel_regularizer=regularizers.l2(0.01)):       
    return Conv2D(ch,(3,3),activation= activation, padding =padding )


In [0]:
#def build_model_1(IMAGE_HEIGHT=IMAGE_HEIGHT,IMAGE_WIDTH=IMAGE_WIDTH,ch=ch):
def build_model_1(ch=8):
    #inputs = Input((IMAGE_HEIGHT,IMAGE_WIDTH,ch))
    inputs = Input((21,21,20))

    conv0 = Conv2D(32,(2,2),padding='valid')(inputs)
    print ('conv0',conv0.get_shape())
    conv1 =  conv_block(ch)(conv0)
    conv1 = conv_block(ch)(conv1)
    pool1 = MaxPooling2D(pool_size=(2,2))(conv1)
    print ('pool1',pool1.get_shape())
    
    conv2 =  conv_block(ch*2)(pool1)
    conv2 = conv_block(ch*2)(conv2)
    pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
    print ('poo12',pool2.get_shape())
    
    conv_test = Conv2D(64,(2,2),padding='valid')(pool2)
    print ('conv_test',conv_test.get_shape())
    
    
    
    conv3 =  conv_block(ch*4)(conv_test)
    conv3 = conv_block(ch*4)(conv3)
    pool3 = MaxPooling2D(pool_size=(2,2))(conv3)
    print ('poo13',pool3.get_shape())
    
    conv4 =  conv_block(ch*8)(pool3)
    conv4 = conv_block(ch*8)(conv4)
    pool4 = MaxPooling2D(pool_size=(2,2))(conv4)
    print ('poo14',pool4.get_shape())
    
    conv5 =  conv_block(ch*16)(pool4)
    conv5 = conv_block(ch*16)(conv5)

    
    up6 = concatenate([UpSampling2D(size=(2,2))(conv5), conv4], axis=3)
    conv6 = conv_block(ch*8)(up6)
    conv6 = conv_block(ch*8)(conv6)
    
    up7 = concatenate ([UpSampling2D(size=(2,2))(conv6), conv3],  axis=3)
    conv7 = conv_block(ch*4)(up7)
    conv7 = conv_block(ch*4)(conv7)
    print ('conv7',conv7.get_shape())
    conv7 = Conv2DTranspose(ch*4,(2,2),padding='valid')(conv7)
    print ('conv7',conv7.get_shape())
    
    up8 = concatenate([UpSampling2D(size=(2,2))(conv7), conv2], axis=3)
    conv8 = conv_block(ch*2)(up8)
    conv8 = conv_block(ch*2)(conv8)
    
    up9 = concatenate([UpSampling2D(size=(2,2))(conv8), conv1],  axis=3)
    conv9 = conv_block(ch)(up9)
    conv9 = conv_block(ch)(conv9)
    
    
    
    conv10 = Conv2D(1,(1,1))(conv9)
    
  
    model = Model(inputs= inputs, outputs=conv10)
    
    return model
    
    

In [9]:
model = build_model_1()
model.summary()

conv0 (?, 20, 20, 32)
pool1 (?, 10, 10, 8)
poo12 (?, 5, 5, 16)
conv_test (?, 4, 4, 64)
poo13 (?, 2, 2, 32)
poo14 (?, 1, 1, 64)
conv7 (?, 4, 4, 32)
conv7 (?, ?, ?, 32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 21, 21, 20)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 20, 20, 32)   2592        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 20, 20, 8)    2312        conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 20

In [0]:



def gram_matrix(x):
  #features = K.batch_flatten(K.permute_dimensions(x[0], (2, 0, 1)))
  x = K.batch_flatten(x)
  gram = K.dot(x, K.transpose(x))
  print(gram)
  #return K.expand_dims(gram, axis=0)
  return gram
  


def my_loss(y_true,y_pred):
  diff_1  = K.mean(K.square(y_pred - y_true), axis=-1)
  diff_2 = K.mean(K.square(gram_matrix(y_pred) - gram_matrix(y_true)), axis=-1)
  
  total_loss = diff_1 * 1.0 + 0.05 * diff_2
  return total_loss
  



In [0]:
model.compile (optimizer='Adam', loss = 'mse', metrics =['mae'])

In [0]:

# train the model on the new data for a few epochs

use_model= False
if use_model:
    

    if os.path.isfile(src+'water_first_try_2.h5'):

      model = load_model(src+'water_first_try_2.h5')
      print ('model laoded')
    else:
      print ('model not exist')


In [13]:

checkpoint = ModelCheckpoint(filepath=src+'water_first_try_2_prepro.h5', monitor = 'val_loss', save_best_only=True, mode= 'auto')
earlystop = EarlyStopping(patience=10)
callback_list = [checkpoint, earlystop]
model.fit(x=train_x, y=train_y, batch_size=32, epochs=300, callbacks = callback_list, verbose=1, validation_split=0.2,  shuffle=True)


Train on 2688 samples, validate on 672 samples
Epoch 1/300
2688/2688 [==============================] - 5s 2ms/step - loss: 1.0956 - mean_absolute_error: 0.8216 - val_loss: 0.6146 - val_mean_absolute_error: 0.6223
Epoch 2/300
2688/2688 [==============================] - 3s 978us/step - loss: 0.5189 - mean_absolute_error: 0.5679 - val_loss: 0.4269 - val_mean_absolute_error: 0.5135
Epoch 3/300
2688/2688 [==============================] - 3s 973us/step - loss: 0.3735 - mean_absolute_error: 0.4776 - val_loss: 0.3256 - val_mean_absolute_error: 0.4457
Epoch 4/300
2688/2688 [==============================] - 3s 992us/step - loss: 0.3001 - mean_absolute_error: 0.4260 - val_loss: 0.2869 - val_mean_absolute_error: 0.4151
Epoch 5/300
2688/2688 [==============================] - 3s 978us/step - loss: 0.2679 - mean_absolute_error: 0.4022 - val_loss: 0.2531 - val_mean_absolute_error: 0.3901
Epoch 6/300
2688/2688 [==============================] - 3s 962us/step - loss: 0.2410 - mean_absolute_error: 0

In [14]:
error = model.evaluate(val_x,val_y)
print (error)

840/840 [==============================] - 0s 303us/step
[0.06548954915432703, 0.19008049737839472]


In [15]:

print ('label mean = ', np.mean(label_data))
print ('per_element_error = ', np.sqrt(error[0]/840.))
print ('error percentage = ', str(round(np.sqrt(error[0]/400.)/np.mean(label_data)*100.,2))+'%')
     

label mean =  1.2508584707231873
per_element_error =  0.008829708318695837
error percentage =  1.02%


In [0]:
!cp drive/aoi_test/t_brain/geo/test2.npy .

In [17]:
print (os.path.isfile('test2.npy'))
test = np.load('test2.npy')
test_data = (test - data_mean)/data_std
test_data.shape

True


(1800, 21, 21, 20)

In [18]:
prediction = model.predict(test_data)
prediction = np.exp(prediction)
pred = prediction.reshape(1800,-1)
print (pred.shape)


pred_df = pd.DataFrame(pred)
pred_df.keys()


(1800, 400)


RangeIndex(start=0, stop=400, step=1)

In [0]:
test_df = pd.read_csv(src+'upload_sample.csv')
col_names = test_df.keys().tolist()

In [20]:
final = pd.concat([test_df['ID'],pred_df],axis=1)
final.columns = col_names
final.to_csv(src+'test_submit_0831.csv', index=False)
print ('submit file saved')

submit file saved


In [21]:
test_see_df = pd.read_csv(src+'test_submit_0831.csv')
test_see_df

,ID,loc1,loc2,loc3,loc4,loc5,loc6,loc7,loc8,loc9,...,loc391,loc392,loc393,loc394,loc395,loc396,loc397,loc398,loc399,loc400
0,F0000009,1.849336,2.120372,0.696887,0.570783,0.588395,0.603600,0.571897,0.735138,1.016605,...,15.718044,6.611536,1.871565,3.115246,3.792983,1.912196,3.079525,4.057917,4.167987,3.303713
1,F0000011,10.120468,9.074798,5.553554,3.543387,2.142448,3.299346,2.315105,2.021475,1.151480,...,4.626652,2.503590,1.610382,1.848381,1.838651,0.759962,0.844151,0.448156,0.361076,0.712740
2,F0000012,3.638349,3.476134,2.297926,1.185128,0.372917,0.345980,0.668110,0.395409,0.426189,...,1.471618,2.833370,2.287346,1.837238,1.078088,0.507185,0.573971,0.908917,1.271319,0.903570
3,F0000014,1.432376,2.189588,1.213771,0.660480,0.219865,0.650551,2.228853,2.289441,1.388398,...,3.031142,2.748054,10.666901,10.877209,12.522958,7.685724,6.422673,4.878351,3.450582,4.106636
4,F0000017,0.166067,0.303007,0.542983,0.744448,0.413350,0.264127,0.919048,1.602198,1.617081,...,9.650559,25.445680,25.854923,24.231780,18.980236,6.232164,2.867024,1.272020,4.136360,2.330391
5,F0000018,4.087423,5.238533,2.610285,1.633973,1.107491,0.462452,0.591882,0.843233,1.268361,...,2.266092,3.253016,1.186978,1.657144,2.818037,4.973001,2.334519,1.828969,4.000922,5.130471
6,F0000024,1.147398,1.152370,1.700854,1.338874,0.969580,0.712712,0.536913,0.683208,0.962251,...,1.019880,1.719711,3.580345,1.947356,1.988458,2.475658,1.363952,1.319875,1.929964,1.892065
7,F0000036,7.184082,8.160756,4.558158,11.040903,7.495656,6.705613,8.612055,5.616785,7.225685,...,16.250862,10.132878,11.404354,21.694635,21.153446,24.701092,30.465046,9.115867,8.364340,10.160685
8,F0000046,7.287852,9.159155,26.335714,17.171846,8.211869,3.623356,1.833662,2.122074,3.290357,...,6.713529,8.038000,6.490035,4.479669,4.444613,8.467362,7.204391,5.312131,5.965121,8.129815
9,F0000047,1.396473,1.514542,1.891761,2.652561,1.673830,1.222837,1.432894,1.293317,1.667840,...,7.142710,11.090627,5.765072,3.629172,4.883206,5.991320,5.064953,5.788856,7.667899,6.364522


In [0]:
!cp drive/aoi_test/t_brain/geo/test_submit.csv .

In [0]:
from google.colab import files
files.download('test_submit.csv')